<a href="https://colab.research.google.com/github/datascience-uniandes/data-quality-tutorial/blob/master/data-quality-tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Quality and Cleanliness

MINE-4101: Applied Data Science  
Univerisdad de los Andes 

Juan Sebastian Parrado - 201531947
  
**Dataset:** Homicides Colombia ([datos.gov.co](datos.gov.co))
  
Last update: September, 2023

In [1]:
!pip install pylev

In [2]:
import re
from random import randint
from datetime import datetime
from difflib import SequenceMatcher

import numpy as np
import pandas as pd

import pylev

In [189]:
pd.set_option("display.max_columns", None)

## 1. Loading the data

In [326]:
homicides_df = pd.read_csv("homicides.csv")

In [327]:
homicides_df.shape

(12400, 22)

In [328]:
homicides_df.dtypes

FECHA                 object
DEPARTAMENTO          object
MUNICIPIO             object
DIA                   object
HORA                  object
BARRIO                object
ZONA                  object
CLASE DE SITIO        object
ARMA O MEDIO          object
MOVIL VICTIMA         object
MOVIL AGRESOR         object
EDAD                 float64
GENERO                object
ESTADO CIVIL          object
CLASE EMPLEADO        object
PROFESION             object
ESCOLARIDAD           object
PAIS NACE             object
CODIGO DANE          float64
AÑO DE NACIMIENTO    float64
CÉDULA                object
CORREO                object
dtype: object

In [329]:
homicides_df.sample(5)

,FECHA,DEPARTAMENTO,MUNICIPIO,DIA,HORA,BARRIO,ZONA,CLASE DE SITIO,ARMA O MEDIO,MOVIL VICTIMA,MOVIL AGRESOR,EDAD,GENERO,ESTADO CIVIL,CLASE EMPLEADO,PROFESION,ESCOLARIDAD,PAIS NACE,CODIGO DANE,AÑO DE NACIMIENTO,CÉDULA,CORREO
1550,02/15/2021 12:00:00 AM,HUILA,ALGECIRAS,Domingo,8:00,VEREDA EL TORO,RURAL,GALLERA,ARMA BLANCA,A PIE,A PIE,36.0,MASCULINO,SOLTERO,AGRICULTOR,NO REPORTADO,PRIMARIA,COLOMBIA,41020000.0,1979.0,33-216,asadmo2358@gmail.com
399,01/10/2021 12:00:00 AM,VALLE,CALI (CT),Sábado,8:05,VALLADO E15,RURAL,LOTE BALDIO,ARMA BLANCA,A PIE,A PIE,14.0,MASCULINO,SOLTERO,DESEMPLEADO,NO REPORTADO,SECUNDARIA,COLOMBIA,76001000.0,2001.0,19-335,koinut6290@unidatos.edu.co
8583,09/13/2021 12:00:00 AM,CUNDINAMARCA,FACATATIVÁ,Domingo,2:00,CENTRO,URBANA,VIAS PUBLICAS,ARMA BLANCA,A PIE,A PIE,26.0,MASCULINO,UNION LIBRE,EMPLEADO PARTICULAR,NO REPORTADO,SECUNDARIA,COLOMBIA,25269000.0,1989.0,47-806,ireepq2256@unidatos.edu.co
6448,07/09/2021 12:00:00 AM,CUNDINAMARCA,FACATATIVÁ,Juees,12:18,SANTA RITA,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,40.0,MASCULINO,UNION LIBRE,COMERCIANTE,NO REPORTADO,SECUNDARIA,COLOMBIA,25269000.0,1975.0,90-176,platap4221@gmail.com
8110,08/29/2021 12:00:00 AM,META,VILLAVICENCIO (CT),Sábado,22:00,LA ESPERANZA,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,36.0,MASCULINO,SOLTERO,INDEPENDIENTE,NO REPORTADO,SECUNDARIA,COLOMBIA,50001000.0,1979.0,71-563,eqkqji4568@gmail.com


## 2. Working with datetimes

In [330]:
# Creating a lambda expression for datetime parsing
dateparse = lambda x: datetime.strptime(x, "%m/%d/%Y %H:%M:%S %p")

In [331]:
# Applying the validation to all values in the column
homicides_df["FECHA"].apply(dateparse)

# IT IS EXPECTED TO HAVE AN ERROR BECAUSE SOME VALUES DOESN'T FIT THE FORMAT

ValueError: time data '13/12/2021 12:00:00 AM' does not match format '%m/%d/%Y %H:%M:%S %p'

In [332]:
# Creating a function for validating which value is causing the previous error
def error_in_format(x):
    try:
        datetime.strptime(x, "%m/%d/%Y %H:%M:%S %p")
        return False
    except:
        return True

In [333]:
# Using the function for validation
homicides_df.loc[homicides_df["FECHA"].apply(error_in_format)]

,FECHA,DEPARTAMENTO,MUNICIPIO,DIA,HORA,BARRIO,ZONA,CLASE DE SITIO,ARMA O MEDIO,MOVIL VICTIMA,MOVIL AGRESOR,EDAD,GENERO,ESTADO CIVIL,CLASE EMPLEADO,PROFESION,ESCOLARIDAD,PAIS NACE,CODIGO DANE,AÑO DE NACIMIENTO,CÉDULA,CORREO
486,13/12/2021 12:00:00 AM,VALLE,CALI (CT),kunes,23:00,MOJICA E15,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,26.0,MASCULINO,SOLTERO,INDEPENDIENTE,NO REPORTADO,SECUNDARIA,COLOMBIA,76001000.0,89.0,80-330,lujhdf9132@gmail.com
695,30/01/2021 12:00:00 AM,BOLÍVAR,CARTAGENA (CT),Lunes,5:30,REP. DEL LIBANO,URBANA,VIAS PUBLICAS,ARMA BLANCA,NO REPORTADO,A PIE,25.0,MASCULINO,SOLTERO,EMPLEADO PARTICULAR,NO REPORTADO,SECUNDARIA,COLOMBIA,13001000.0,1990.0,12-915,ghumtg4094@unidatos.edu.co
1250,18/05/2021 12:00:00 AM,HUILA,TESALIA,Jueves,19:30,VEREDA PACARNI,RURAL,CASAS DE HABITACION,ARMA DE FUEGO,A PIE,A PIE,34.0,FEMENINO,SOLTERO,AGRICULTOR,NO REPORTADO,PRIMARIA,COLOMBIA,41797000.0,1981.0,99-095,sdaggf6639@gmail.com
12168,12/25/2021 12:00:00 MM,VALLE,PALMIRA,Viernes,15:30,LA EMILIA,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,CONDUCTOR MOTOCICLETA,17.0,MASCULINO,SOLTERO,EMPLEADO PARTICULAR,NO REPORTADO,SECUNDARIA,COLOMBIA,76520000.0,1998.0,16-362,uschca1775@gmail.com
12399,TOTAL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [334]:
filtered_df = homicides_df.loc[homicides_df["FECHA"].apply(error_in_format)]
dimensions = filtered_df.shape

print("Number of rows:", dimensions[0])
print("Number of columns:", dimensions[1])

Number of rows: 5
Number of columns: 22


In [335]:
# Deleting a row by its index
homicides_df.drop([486, 695, 1250, 12168, 12399], inplace=True)

In [336]:
# Trying to parse the datetime string again
homicides_df["FECHA"] = homicides_df["FECHA"].apply(dateparse)

In [337]:
homicides_df.dtypes

FECHA                datetime64[ns]
DEPARTAMENTO                 object
MUNICIPIO                    object
DIA                          object
HORA                         object
BARRIO                       object
ZONA                         object
CLASE DE SITIO               object
ARMA O MEDIO                 object
MOVIL VICTIMA                object
MOVIL AGRESOR                object
EDAD                        float64
GENERO                       object
ESTADO CIVIL                 object
CLASE EMPLEADO               object
PROFESION                    object
ESCOLARIDAD                  object
PAIS NACE                    object
CODIGO DANE                 float64
AÑO DE NACIMIENTO           float64
CÉDULA                       object
CORREO                       object
dtype: object

In [338]:
# Counting homicides by hour
homicides_df["FECHA"].dt.hour.value_counts()

# All datetime hour parts are the same

12    12395
Name: FECHA, dtype: int64

*The homicide hour is available in a different column!*

In [339]:
# Merging both columns
homicides_df["FECHA"] = homicides_df["FECHA"].astype(str).apply(lambda x: x[:11])+homicides_df["HORA"]

In [340]:
# Deleting redundant column
homicides_df.drop(columns=["HORA"], inplace=True)

In [341]:
# Making a new expression for datetime parsing
dateparse = lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M")

In [342]:
# Applying the expression
homicides_df["FECHA"] = homicides_df["FECHA"].apply(dateparse)

In [343]:
homicides_df.dtypes

FECHA                datetime64[ns]
DEPARTAMENTO                 object
MUNICIPIO                    object
DIA                          object
BARRIO                       object
ZONA                         object
CLASE DE SITIO               object
ARMA O MEDIO                 object
MOVIL VICTIMA                object
MOVIL AGRESOR                object
EDAD                        float64
GENERO                       object
ESTADO CIVIL                 object
CLASE EMPLEADO               object
PROFESION                    object
ESCOLARIDAD                  object
PAIS NACE                    object
CODIGO DANE                 float64
AÑO DE NACIMIENTO           float64
CÉDULA                       object
CORREO                       object
dtype: object

In [344]:
homicides_df.sample(5)

,FECHA,DEPARTAMENTO,MUNICIPIO,DIA,BARRIO,ZONA,CLASE DE SITIO,ARMA O MEDIO,MOVIL VICTIMA,MOVIL AGRESOR,EDAD,GENERO,ESTADO CIVIL,CLASE EMPLEADO,PROFESION,ESCOLARIDAD,PAIS NACE,CODIGO DANE,AÑO DE NACIMIENTO,CÉDULA,CORREO
8637,2021-09-14 12:32:00,VALLE,TULUÁ,Lunes,Riopaila,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,CONDUCTOR MOTOCICLETA,41.0,MASCULINO,UNION LIBRE,EMPLEADO PARTICULAR,NO REPORTADO,SECUNDARIA,COLOMBIA,76834000.0,1974.0,80-394,jrossr8696@gmail.com
7303,2021-08-03 09:20:00,N. DE SANTANDER,LA ESPERANZA,Lunes,VEREDA CAÑO DE HOYO,RURAL,VIAS PUBLICAS,ARMA DE FUEGO,PASAJERO MOTOCICLETA,VEHICULO,34.0,MASCULINO,SOLTERO,DESEMPLEADO,NO REPORTADO,PRIMARIA,COLOMBIA,54385000.0,1981.0,96-318,correo5241@colombia.gov.co
2456,2021-03-16 15:00:00,ANTIOQUIA,GUARNE,Lunes,ROMERAL,RURAL,FINCAS Y SIMILARES,BOLSA PLASTICA,A PIE,A PIE,30.0,MASCULINO,UNION LIBRE,EMPLEADO PARTICULAR,NO REPORTADO,SECUNDARIA,COLOMBIA,5318000.0,1985.0,84-001,eitjan1737@gmail.com
426,2021-01-11 05:59:00,CUNDINAMARCA,GIRARDOT,domnigo,CENTRO,URBANA,VIAS PUBLICAS,ARMA BLANCA,A PIE,A PIE,21.0,MASCULINO,SOLTERO,DESEMPLEADO,NO REPORTADO,PRIMARIA,COLOMBIA,25307000.0,1994.0,69-341,fagdff4429@unidatos.edu.co
6794,2021-07-19 21:30:00,META,GRANADA,Domingo,VILLA ESPERANZA,URBANA,LOTE BALDIO,ARMA DE FUEGO,A PIE,A PIE,28.0,MASCULINO,SOLTERO,INDEPENDIENTE,NO REPORTADO,ANALFABETA,COLOMBIA,50313000.0,1987.0,45-800,correo3262@colombia.gov.co


## 2. Fixing categorical column

In [345]:
# Creating a dictionary representing the valid departments for Colombia
departments_list = ['ANTIOQUIA', 'ATLÁNTICO', 'BOLÍVAR', 'BOYACÁ', 'CALDAS', 'CAQUETÁ',
       'CASANARE', 'CAUCA', 'CESAR', 'CHOCÓ', 'CÓRDOBA', 'META',
       'CUNDINAMARCA', 'HUILA', 'MAGDALENA', 'NARIÑO', 'PUTUMAYO',
       'RISARALDA', 'SANTANDER', 'SUCRE', 'TOLIMA', 'VALLE',
       'NORTE DE SANTANDER', 'GUAJIRA', 'QUINDÍO', 'SAN ANDRÉS Y PROVIDENCIA', 'ARAUCA',
       'GUAINÍA', 'VICHADA', 'VAUPÉS', 'GUAVIARE', 'AMAZONAS']

In [346]:
# Finding values not matching with the dictionary
homicides_df.loc[~homicides_df["DEPARTAMENTO"].isin(departments_list), "DEPARTAMENTO"].unique()

array(['SAN ANDRÉS', 'N. DE SANTANDER'], dtype=object)

In [347]:
filtered_series = homicides_df.loc[~homicides_df["DEPARTAMENTO"].isin(departments_list), "DEPARTAMENTO"]
dimensions = filtered_series.shape
print("Number of rows:", dimensions[0])

Number of rows: 101


<span style="color:red">TODO: Replace the values identified as error to a valid value from the dictionary.</span>

<span style="color:red">Hint: You can use the replace() pandas function.</span>

In [348]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

# Función para encontrar el mejor ajuste usando Levenshtein
def find_best_match(name, choices):
    return process.extractOne(name, choices, scorer=fuzz.ratio)

# Aplica la función a cada valor en la columna neighbourhood_cleansed
homicides_df['DEPARTAMENTO_AJUSTADO'] = homicides_df['DEPARTAMENTO'].apply(
    lambda x: find_best_match(x, departments_list)[0]
)

# Imprime el DataFrame actualizado
unique_categories = homicides_df['DEPARTAMENTO_AJUSTADO'].unique()
print(unique_categories)
len(unique_categories)

['ANTIOQUIA' 'ATLÁNTICO' 'BOLÍVAR' 'BOYACÁ' 'CALDAS' 'CAQUETÁ' 'CASANARE'
 'CAUCA' 'CESAR' 'CHOCÓ' 'CÓRDOBA' 'META' 'CUNDINAMARCA' 'HUILA'
 'MAGDALENA' 'NARIÑO' 'PUTUMAYO' 'RISARALDA' 'SANTANDER' 'SUCRE' 'TOLIMA'
 'VALLE' 'NORTE DE SANTANDER' 'GUAJIRA' 'QUINDÍO' 'ARAUCA' 'GUAINÍA'
 'VICHADA' 'GUAVIARE' 'VAUPÉS' 'SAN ANDRÉS Y PROVIDENCIA' 'AMAZONAS']


32

## 3. Analyzing potential duplicates

In [349]:
# Detecting duplicates by "CÉDULA" column
duplicates_by_cedula = homicides_df.loc[homicides_df["CÉDULA"].duplicated(keep=False)]

In [350]:
duplicates_by_cedula.shape

(1627, 22)

In [351]:
duplicates_by_cedula.sort_values("CÉDULA", ascending=True).head(6)

,FECHA,DEPARTAMENTO,MUNICIPIO,DIA,BARRIO,ZONA,CLASE DE SITIO,ARMA O MEDIO,MOVIL VICTIMA,MOVIL AGRESOR,EDAD,GENERO,ESTADO CIVIL,CLASE EMPLEADO,PROFESION,ESCOLARIDAD,PAIS NACE,CODIGO DANE,AÑO DE NACIMIENTO,CÉDULA,CORREO,DEPARTAMENTO_AJUSTADO
4041,2021-05-03 23:30:00,CAUCA,EL TAMBO,Domingo,LA VICTORIA,RURAL,"BARES, CANTINAS Y SIMILARES",ARMA DE FUEGO,A PIE,A PIE,45.0,MASCULINO,UNION LIBRE,AGRICULTOR,NO REPORTADO,PRIMARIA,COLOMBIA,19256000.0,1970.0,10-048,aitufn1227@gmail.com,CAUCA
3810,2021-04-26 03:00:00,ANTIOQUIA,SAN JERÓNIMO,Domingo,LA PLAYA,URBANA,"HOTELES, RESIDENCIAS, Y SIMILARES.",CUERDA/SOGA/CADENA,A PIE,A PIE,25.0,MASCULINO,SOLTERO,INDEPENDIENTE,NO REPORTADO,SECUNDARIA,COLOMBIA,5656000.0,1990.0,10-048,ibbcpu2509@unidatos.edu.co,ANTIOQUIA
693,2021-01-19 05:30:00,ANTIOQUIA,SALGAR,Lunes,LA HABANA,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,58.0,MASCULINO,SOLTERO,AGRICULTOR,NO REPORTADO,PRIMARIA,COLOMBIA,5642000.0,57.0,10-079,unhoqj1172@unidatos.edu.co,ANTIOQUIA
10796,2021-11-20 03:00:00,VALLE,CALI (CT),Viernes,POTRERO GRANDE E21,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,21.0,MASCULINO,SOLTERO,EMPLEADO PARTICULAR,NO REPORTADO,SECUNDARIA,COLOMBIA,76001000.0,1994.0,10-079,iknnoj8430@unidatos.edu.co,VALLE
2386,2021-03-14 02:00:00,META,FUENTE DE ORO,Sábado,VEREDA PUERTO NUEVO,RURAL,"BARES, CANTINAS Y SIMILARES",ARMA BLANCA,A PIE,A PIE,42.0,MASCULINO,SOLTERO,INDEPENDIENTE,NO REPORTADO,PRIMARIA,COLOMBIA,50287000.0,1973.0,10-255,afompq7113@unidatos.edu.co,META
138,2021-01-02 06:20:00,CAUCA,SOTARA,Viernes,CENTRO,RURAL,VIAS PUBLICAS,ARMA BLANCA,A PIE,A PIE,22.0,MASCULINO,SOLTERO,EMPLEADO EJERCITO,NO REPORTADO,SECUNDARIA,COLOMBIA,19760000.0,1993.0,10-255,cobgqs8819@unidatos.edu.co,CAUCA


<span style="color:red">TODO: Delete records with "CÉDULA" duplicated.</span>

<span style="color:red">Hint: You can use drop_duplicates() pandas function.</span>

In [352]:
homicides_df=homicides_df.drop_duplicates(['CÉDULA'])

In [353]:
# Detecting duplicates by "CÉDULA" column
duplicates_by_cedula = homicides_df.loc[homicides_df["CÉDULA"].duplicated(keep=False)]

In [354]:
duplicates_by_cedula.shape

(0, 22)

## 4. Fixing formats

In [355]:
# Using regular expressions for validating if "CÉDULA" values match the pattern XX-XXX
cedula_malformed = homicides_df.loc[homicides_df["CÉDULA"].apply(lambda x: (re.match("\d{2}-\d{3}", x) is None))]

In [356]:
cedula_malformed.shape

(28, 22)

In [357]:
cedula_malformed.head(6)

,FECHA,DEPARTAMENTO,MUNICIPIO,DIA,BARRIO,ZONA,CLASE DE SITIO,ARMA O MEDIO,MOVIL VICTIMA,MOVIL AGRESOR,EDAD,GENERO,ESTADO CIVIL,CLASE EMPLEADO,PROFESION,ESCOLARIDAD,PAIS NACE,CODIGO DANE,AÑO DE NACIMIENTO,CÉDULA,CORREO,DEPARTAMENTO_AJUSTADO
172,2021-01-03 11:00:00,META,MESETAS,Sábado,VEREDA EL CAFRE,RURAL,ZONA SELVÁTICA,MINA ANTIPERSONA,A PIE,A PIE,24.0,MASCULINO,SOLTERO,EMPLEADO EJERCITO,NO REPORTADO,SECUNDARIA,COLOMBIA,50330000.0,1991.0,680-21,oohghd8899@gmail.com,META
1114,2021-02-01 12:30:00,ATLÁNTICO,BARRANQUILLA (CT),Domingo,LA LUZ,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,29.0,MASCULINO,SOLTERO,INDEPENDIENTE,NO REPORTADO,PRIMARIA,COLOMBIA,8001000.0,1986.0,140-17,correo5853@colombia.gov.co,ATLÁNTICO
2119,2021-03-06 15:30:00,VALLE,CALI (CT),Viernes,QUINTAS DEL SOL E14,URBANA,DENTRO DE LA VIVIENDA,ARMA BLANCA,A PIE,A PIE,20.0,FEMENINO,UNION LIBRE,EMPLEADO PARTICULAR,NO REPORTADO,SECUNDARIA,COLOMBIA,76001000.0,1995.0,975-31,ohbqrk3631@unidatos.edu.co,VALLE
3309,2021-04-11 10:30:00,CAQUETÁ,FLORENCIA (CT),Sávado,VIA MORELIA,RURAL,CARCELES,CORTANTES,A PIE,A PIE,23.0,MASCULINO,SOLTERO,INDEPENDIENTE,NO REPORTADO,PRIMARIA,COLOMBIA,18001000.0,1992.0,348-66,pdkqur8407@unidatos.edu.co,CAQUETÁ
3409,2021-04-13 23:20:00,CÓRDOBA,SAHAGÚN,Lunes,CORREGIMIENTO DE BAJO GRANDE,URBANA,BILLARES,CONTUNDENTES,A PIE,A PIE,49.0,MASCULINO,SOLTERO,INDEPENDIENTE,NO REPORTADO,SECUNDARIA,COLOMBIA,23660000.0,1966.0,496-18,fdbbeo6751@unidatos.edu.co,CÓRDOBA
3588,2021-04-19 20:00:00,BOLÍVAR,CARTAGENA (CT),Domingo,LA ESPERANZA,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,45.0,MASCULINO,SOLTERO,DESEMPLEADO,NO REPORTADO,NO REPORTADO,NO REPORTADO,13001000.0,1970.0,188-03,diebuo5651@unidatos.edu.co,BOLÍVAR


<span style="color:red">TODO: Fix the malformed "CÉDULA" values.</span>

In [358]:
homicides_df['CÉDULA'] = homicides_df['CÉDULA'].str.replace(' ', '')
homicides_df['CÉDULA'] = homicides_df['CÉDULA'].str.replace('-', '')
# Insert hyphen after the first two digits
homicides_df['CÉDULA'] = homicides_df['CÉDULA'].str.replace(r'^(\d{2})(.*)$', r'\1-\2', regex=True)




# Using regular expressions for validating if "CÉDULA" values match the pattern XX-XXX
cedula_malformed = homicides_df.loc[homicides_df["CÉDULA"].apply(lambda x: (re.match("\d{2}-\d{3}", x) is None))]

cedula_malformed.shape

(0, 22)

In [359]:
cedula_malformed.head(6)

,FECHA,DEPARTAMENTO,MUNICIPIO,DIA,BARRIO,ZONA,CLASE DE SITIO,ARMA O MEDIO,MOVIL VICTIMA,MOVIL AGRESOR,EDAD,GENERO,ESTADO CIVIL,CLASE EMPLEADO,PROFESION,ESCOLARIDAD,PAIS NACE,CODIGO DANE,AÑO DE NACIMIENTO,CÉDULA,CORREO,DEPARTAMENTO_AJUSTADO


<span style="color:red">TODO: Make something similar to check and fix the "CORREO" column (PATTERN: 4 digits before the @, only .edu.co and .com domains are allowed)</span>

In [360]:
correo_malformed = homicides_df.loc[homicides_df["CORREO"].apply(lambda x: (re.match("\w+\d{4}@\w+(\.edu.co|\.com)", x) is None))]

In [361]:
correo_malformed.shape

(2134, 22)

In [362]:
correo_malformed.head(6)

,FECHA,DEPARTAMENTO,MUNICIPIO,DIA,BARRIO,ZONA,CLASE DE SITIO,ARMA O MEDIO,MOVIL VICTIMA,MOVIL AGRESOR,EDAD,GENERO,ESTADO CIVIL,CLASE EMPLEADO,PROFESION,ESCOLARIDAD,PAIS NACE,CODIGO DANE,AÑO DE NACIMIENTO,CÉDULA,CORREO,DEPARTAMENTO_AJUSTADO
28,2021-01-01 15:00:00,CAQUETÁ,SAN JOSÉ DEL FRAGUA,Jueces,CENTRO,URBANA,DISCOTECAS,ARMA DE FUEGO,A PIE,A PIE,24.0,MASCULINO,UNION LIBRE,INDEPENDIENTE,NO REPORTADO,PRIMARIA,COLOMBIA,18610000.0,1991.0,94-565,correo975@unidatos.edu.co,CAQUETÁ
36,2021-01-01 08:00:00,CAUCA,PATÍA,Jueves,C/MIENTO PIEDRASENTADA,RURAL,CARRETERA,ARMA DE FUEGO,VEHICULO,A PIE,26.0,MASCULINO,SOLTERO,EMPLEADO EJERCITO,NO REPORTADO,SECUNDARIA,COLOMBIA,19532000.0,1989.0,69-084,correo4714@colombia.gov.co,CAUCA
47,2021-01-01 09:00:00,CHOCÓ,TADÓ,Jueves,POPULAR,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,34.0,MASCULINO,CASADO,EMPLEADO POLICIAL,POLICIA,TECNICO,COLOMBIA,27787000.0,1981.0,29-957,correo8297@colombia.gov.co,CHOCÓ
55,2021-01-01 01:30:00,CUNDINAMARCA,BOGOTÁ D.C. (CT),Jueves,ÁLVARO BERNAL SEGURA E-19,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,25.0,MASCULINO,UNION LIBRE,DESEMPLEADO,NO REPORTADO,SECUNDARIA,COLOMBIA,11001000.0,1990.0,35-260,correo7285@colombia.gov.co,CUNDINAMARCA
60,2021-01-01 09:50:00,CUNDINAMARCA,BOGOTÁ D.C. (CT),Jueves,LAS COLINAS E-18,URBANA,VIAS PUBLICAS,ARMA BLANCA,A PIE,A PIE,37.0,MASCULINO,SOLTERO,INDEPENDIENTE,NO REPORTADO,SECUNDARIA,COLOMBIA,11001000.0,1978.0,96-322,correo870@unidatos.edu.co,CUNDINAMARCA
63,2021-01-01 17:53:00,CUNDINAMARCA,BOGOTÁ D.C. (CT),Jueces,LA ESTRELLA SECTOR LAGOS E-19,URBANA,VIAS PUBLICAS,ARMA BLANCA,A PIE,A PIE,19.0,MASCULINO,SOLTERO,INDEPENDIENTE,NO REPORTADO,SECUNDARIA,COLOMBIA,11001000.0,1996.0,45-426,correo102@unidatos.edu.co,CUNDINAMARCA


## 5. Automating imputation of categorical values

In [363]:
homicides_df["DIA"].unique()

array(['Jueves', 'Juees', 'Jueces', 'juves', 'Juevrs', 'Viernes',
       'Viermes', 'iernes', 'virnes', 'Vierens', 'Sábado', 'Sabadi',
       'Sabado', 'sábad', 'Sávado', 'Ssbado', 'Domingo', 'Domungo',
       'Doningo', 'domungo', 'Lunes', 'lune', 'Luns', 'Lumes', 'kunes',
       'Lnues', 'Martes', 'Mates', 'Marte', 'mates', 'Miércoles',
       'Miwrcoles', 'Mircoles', 'Voernes', 'domnigo', 'Maryes',
       'Miercoles', 'miércles', 'Dominog', 'Msrtes', 'Mirrcoles'],
      dtype=object)

In [364]:
# Calculating the distance between two words using the Levenshtein distance
pylev.levenshtein("sábado", "sabaod")

3

In [365]:
pylev.levenshtein("sábado", "viernes")

7

In [366]:
SequenceMatcher(None, "sábado", "sabaod").ratio()

0.6666666666666666

In [367]:
SequenceMatcher(None, "sábado", "viernes").ratio()

0.15384615384615385

<span style="color:red">How does SequenceMatcher works? How this differ from the Levenshtein distance?</span>

SequenceMatcher busca secuencias continuas de caracteres que coincidan entre dos textos, mientras que la distancia de Levenshtein evalúa las coincidencias uno a uno, caracter por caracter, para determinar la similitud entre los textos. En otras palabras, SequenceMatcher se enfoca en encontrar partes continuas que se parezcan al estandar de los textos, mientras que la distancia de Levenshtein analiza las similitudes y diferencias entre los caracteres individuales en ambos textos.

<span style="color:red">TODO: Create a function to fix the digitation errors for the "DIA" column.</span>

In [368]:
array=['Lunes','Martes','Miércoles','Jueves','Viernes','Sabado','Domingo']

def find_best_match(value):
    return max(array, key=lambda x: SequenceMatcher(None, value, x).ratio())

homicides_df["DIA"] = homicides_df["DIA"].apply(find_best_match)

In [369]:
homicides_df["DIA"].unique()

array(['Jueves', 'Viernes', 'Sabado', 'Domingo', 'Lunes', 'Martes',
       'Miércoles'], dtype=object)

## 6. Recalculation based on a different column

In [370]:
homicides_df[["AÑO DE NACIMIENTO", "EDAD"]].sample(10)

,AÑO DE NACIMIENTO,EDAD
3604,2012.0,3.0
3020,1987.0,28.0
12346,1991.0,24.0
5344,1991.0,24.0
1407,1998.0,17.0
12321,1973.0,42.0
11293,1997.0,18.0
8337,93.0,22.0
11824,1967.0,48.0
2372,1997.0,18.0


<span style="color:red">TODO: Fix the "AÑO DE NACIMIENTO" column using the column "EDAD".</span>

In [371]:
def update_birth_year(row):
    current_year = datetime.now().year
    birth_year = current_year - row['EDAD']
    return birth_year

homicides_df['AÑO DE NACIMIENTO AJUSTADO'] = homicides_df.apply(update_birth_year, axis=1)

# Comparar las columnas y obtener una matriz de diferencias
discrepancias_matrix = (homicides_df['AÑO DE NACIMIENTO'] != homicides_df['AÑO DE NACIMIENTO AJUSTADO']).values


# Convertir la matriz de diferencias en una matriz NumPy
discrepancias_matrix_np = np.array(discrepancias_matrix, dtype=int)

# Contar las discrepancias
numero_discrepancias = np.count_nonzero(discrepancias_matrix_np)

print(numero_discrepancias)

11559


In [372]:
def update_birth_year(row):
    current_year = datetime.now().year
    birth_year = current_year - row['EDAD']
    return birth_year

homicides_df['AÑO DE NACIMIENTO'] = homicides_df.apply(update_birth_year, axis=1)

In [373]:
homicides_df["AÑO DE NACIMIENTO"].unique()

array([1979., 1993., 1990., 1983., 1957., 1981., 1998., 1986., 1982.,
       1994., 2002., 2003., 1991., 1984., 2006., 1995., 2005., 2000.,
       1985., 1973., 1999., 1988., 1997., 2001., 1989., 1992., 1965.,
       1996., 2008., 2004., 1980., 1987., 1966., 1968., 1956., 1977.,
       1953., 1958., 1974., 2007., 1970., 2009., 1959., 1975., 1972.,
       1976., 1948., 1971., 1969., 1961., 1978., 1964., 1967., 2022.,
       1949., 1940., 1943., 1946., 1955., 1947., 1962., 2023., 1945.,
       2012., 2010., 2017., 1952., 2018., 1942., 1960., 2015., 2019.,
       2013., 1950., 1944., 1933., 1963., 2021., 2014., 1939., 1951.,
       2011., 1954., 1941., 2020., 1937., 2016., 1938., 1931., 1934.])

## 7. Conclusion

<span style="color:red">Make a summary of the different data quality problems found on the dataset, the data quality dimension that is related to and the implemented strategy for solving or mitigating that specific problem.</span>

**1. Working with datetimes:** 

En esta primera etapa del análisis, observamos que la columna 'fecha' está registrada como un tipo de dato 'objeto', lo que incumple con el principio de consistencia de datos. Según las mejores prácticas, esta columna debería ser de tipo 'date' para garantizar la uniformidad en la presentación de la información. A nivel de atributos, procedemos a evaluar los valores que no cumplen con la sintaxis esperada, lo que resulta en una tabla con las siguientes dimensiones:

Número de filas: 5
Número de columnas: 22

Es relevante destacar que la cantidad de registros que no cumplen con el estándar es minima en relación al total de registros iniciales (5 de 12,400, es decir, un 0.04%). Por lo tanto, se procede a eliminar los registros que no cumplen con el formato establecido para mantener la integridad de los datos

**2. Fixing categorical column:** 

En esta segunda fase del análisis, hemos identificado una falta de conformidad en los departamentos enlistados, ya que sus registros no correspondían al estándar requerido para la presentación de la información. Se identificaron 101 registros que no cumplían con los requisitos, lo que resultó en una tabla con las siguientes dimensiones:

Número de filas: 101
Número de columnas: 22

Para abordar los errores de digitación a nivel del atributo de la columna, hemos utilizado la distancia de Levenshtein para corregir las discrepancias y garantizar la integridad de los datos.

**3. Analyzing potential duplicates:** 

En esta tercera fase del análisis, hemos detectado la existencia de registros duplicados, lo que representa una violación del principio de 'Unicidad' o 'No duplicación' en la calidad de la información, específicamente en el atributo 'Cedula'. Este hallazgo ha dado lugar a una tabla con las siguientes dimensiones:

Número de filas: 1627
Número de columnas: 22

Con el propósito de abordar esta problemática, hemos empleado la función 'drop' para eliminar estos registros duplicados y así garantizar la integridad y calidad de los datos.

**4. Automating imputation of categorical values:** 

En esta cuarta etapa del análisis, hemos identificado una falta de conformidad en los números de cédula, dado que los registros no cumplen con el estándar requerido para la presentación de la información. En total, se encontraron 28 registros que no se ajustaban a los requisitos, lo que resultó en una tabla con las siguientes dimensiones:

Número de filas: 28
Número de columnas: 22

Para abordar esta problemática, hemos llevado a cabo una serie de correcciones. Primero, eliminamos los caracteres '-' y los espacios que pudieran estar presentes en los registros, y posteriormente ajustamos su formato a 'XX-XXX' al insertar el carácter '-' después de los primeros dos dígitos. Estas modificaciones han permitido resolver satisfactoriamente el problema identificado y asegurar la coherencia en la presentación de los datos.

Además, con el objetivo de identificar aquellos valores que no cumplían con el principio de relevancia al no ser pertinentes para su inclusión en el conjunto de datos, se llevó a cabo la búsqueda de aquellos registros que no cumplían con el formato requerido: '4 dígitos antes de la @, solo dominios .edu.co y .com'. Este proceso se basó en la aplicación de un patrón de búsqueda que identificaba los valores que no se ajustaban a este estándar: '\w+\d{4}@\w+(.edu.co|.com)', lo que resultó en una tabla con las siguientes dimensiones:

Número de filas: 2134
Número de columnas: 22

**5. Recalculation based on a different column:** 

En esta fase final del análisis, se detectó una falta de conformidad que afectaba el principio de temporalidad, ya que la columna 'AÑO DE NACIMIENTO' no representaba de manera precisa el año de nacimiento de cada individuo. Este hallazgo resultó en una tabla con las siguientes dimensiones:

Número de filas: 11,559
Número de columnas: 22

Es importante destacar que se utilizó la columna 'Edad' para calcular el año de nacimiento en lugar de depender únicamente de la columna 'AÑO DE NACIMIENTO', lo que reveló discrepancias entre ambas coulmnas(11,559). Para resolver esta problemática, se aplicó un enfoque que consistió en restar el año actual a la edad de cada persona para calcular su año de nacimiento, obteniendo así resultados lógicos y ajustados a la realidad.